In [1]:
import numpy as np
import copy

In [2]:
### Something that reacts to a signal
class SignalSensitive:
  # Gets triggered when the `at_index`-th
  # input of a component
  # receives a signal
  def on_input_changed(self, at_index, signal):
    raise NotImplementedError()

### Info about an outgoing connection
### The component is connected to the `at_index`-th input
### of `to_component`
class ConnectionInfo:
  # to_index        :: Int
  # to_component    :: Component -- or SignalSensitive
  def __init__(self, to_index, to_component):
    self.to_index = to_index
    self.to_component = to_component

  def notify(self, signal):
    if self.to_component == None:
      return
    self.to_component.on_input_changed(self.to_index,
                                       signal)

### Placeholder for empty slots
NoConnection = ConnectionInfo(to_index=-1,
                              to_component=None)

In [3]:
class Component(SignalSensitive):
  # n_input_slots                 :: Int       
  # n_output_slots                :: Int
  # input_slots                   :: List<Signal>
  # output_slots                  :: List<Signal>
  # connections                   :: List<ConnectionInfo>
  def __init__(self,
               n_input_slots,
               n_output_slots,
               name='Unknown'):
    self.n_input_slots = n_input_slots
    self.n_output_slots = n_output_slots
    self.input_slots = [False for _ in range(n_input_slots)]
    self.output_slots = [False for _ in range(n_output_slots)]
    self.connections = [NoConnection for _ in range(n_output_slots)]  
    self.input_connections = [None for _ in range(n_input_slots)]

  def connect_at_index(self, my_output_index, neighbor_input_index, neighbor):
    if self.connections[my_output_index] != NoConnection:
      raise ValueError("This slot is already connected to something")
    
    self.connections[my_output_index] = ConnectionInfo(to_index=neighbor_input_index,
                                                       to_component=neighbor)
    neighbor.input_connections[neighbor_input_index] = self    
    self.connections[my_output_index].notify(self.output_slots[my_output_index])  

  def disconnect_at_index(self, my_output_index):
    ### Sends an empty signal
    self.connections[my_output_index].notify(False)
    self.connections[my_output_index] = NoConnection

In [4]:
class Wire(Component):
  def __init__(self):
    super(Wire, self).__init__(n_input_slots=1,
                               n_output_slots=1)

  def resize(self, n):
    if n == self.n_output_slots:
      return
    if n <= 0:
      n = 1
    if n < self.n_output_slots:
      self.output_slots = self.output_slots[:n]
      ## TODO: Traverse from n-th and send empty signal
    else:
      self.output_slots = self.output_slots + [False for _ in range(n - self.n_output_slots)]
    self.n_output_slots = n
  
  def on_input_changed(self, at_index, signal):
    if self.input_slots[at_index] == signal:
      return
    
    self.input_slots[at_index] = signal
    for conn in self.connections:
      conn.notify(signal)
  
  def serialize(self):
    return self.input_connections[0].serialize()

  def __str__(self):
    return f'A wire with {self.n_output_slots} output slots.'


In [5]:
### Two inputs, one output
class BinaryGate(Component):
  # op :: (Signal, Signal) -> Signal
  def __init__(self, op):
    super(BinaryGate, self).__init__(n_input_slots=2,
                                     n_output_slots=1)
    self.op = op

  def i1(self):
    return self.input_slots[0]

  def i2(self):
    return self.input_slots[1]

  def o(self):
    return self.output_slots[0]

  # wrapper
  def connect_output(self, to_index, to_component):
    self.connect_at_index(0, to_index, to_component)

  def on_input_changed(self, at_index, signal):
    if self.input_slots[at_index] == signal:
      return
    self.input_slots[at_index] = signal
    new_output = self.op(self.i1(), self.i2())
    if new_output == self.o():
      return
    self.output_slots[0] = new_output
    for conn in self.connections:
      conn.notify(new_output)


In [6]:
class AndGate(BinaryGate):
  def __init__(self):
    super(AndGate, self).__init__(op = lambda a, b: a and b)

  def serialize(self):
    return self.input_connections[0].serialize() + " && " + self.input_connections[1].serialize()
  


class OrGate(BinaryGate):
  def __init__(self):
    super(OrGate, self).__init__(op = lambda a, b: a or b)

class XorGate(BinaryGate):
  def __init__(self):
    super(XorGate, self).__init__(op = lambda a, b: a != b)

In [7]:
class Pin(Component):
  def __init__(self, val=False):
    super(Pin, self).__init__(n_input_slots=0,
                              n_output_slots=1)
    self.output_slots[0] = val
  
  def flip(self):
    self.output_slots[0] = not self.output_slots[0]
    for conn in self.connections:
      conn.notify(self.output_slots[0])

  def serialize(self):
    return self.name
  
  def set_val(self, val):
    if self.output_slots[0] == val:
      return
    self.flip()

In [8]:
class LightBulb(Component):
  def __init__(self):
    super(LightBulb, self).__init__(n_input_slots=1,
                                    n_output_slots=1)
  
  def connect_at_index(self, at_index, to_component):
    raise ValueError("Lightbulb cannot be connected to anything")

  def serialize(self):
    return self.input_connections[0].serialize()

  def on_input_changed(self, at_index, signal):
    self.input_slots[at_index] = signal
    self.output_slots[at_index] = signal

In [9]:
class CompositeComponent(Component):
  def __init__(self, pins, lightbulbs):
    super(CompositeComponent, self).__init__(n_input_slots=len(pins),
                                             n_output_slots=len(lightbulbs))
    self.pins = [copy.deepcopy(p) for p in pins]
    self.lightbulbs = [copy.deepcopy(l) for l in lightbulbs]
    # self.internal_components = internal_components

  def flip_pin(self, at_index):
    self.pins[at_index].flip()

  def get_output(self, at_index):
    return self.lightbulbs[at_index].output_slots[0]

![image.png](https://i.stack.imgur.com/TpBpr.gif)

In [10]:
A0 = Pin(False)
A0.name = 'A0'
A1 = Pin(False)
A1.name = 'A1'

and1 = AndGate()

w1 = Wire()
w1.resize(2)

w2 = Wire()
w2.resize(2)

A0.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=w1)

A1.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=w2)

w1.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=and1)

w2.connect_at_index(my_output_index=0,
                    neighbor_input_index=1,
                    neighbor=and1)

lb = LightBulb()
w3 = Wire()
and1.connect_at_index(0, 0, w3)
w3.connect_at_index(0, 0, lb)

print('AND1: ', and1.output_slots)
print('LB: ', lb.output_slots)
A0.flip()
print('AND1: ', and1.output_slots)
print('LB: ', lb.output_slots)
A1.flip()
print('AND1: ', and1.output_slots)
print('LB: ', lb.output_slots)

cc = CompositeComponent(pins=[A0, A1], 
                        lightbulbs= [lb])


AND1:  [False]
LB:  [False]
AND1:  [False]
LB:  [False]
AND1:  [True]
LB:  [True]


In [11]:
print(f'{cc.pins[0].output_slots[0]} xor {cc.pins[1].output_slots[0]} = {cc.get_output(0)}')
cc.flip_pin(0)
print(f'{cc.pins[0].output_slots[0]} xor {cc.pins[1].output_slots[0]} = {cc.get_output(0)}')

True xor True = True
False xor True = True


In [12]:
lb.serialize()

'A0 && A1'